In [1]:
import scanpy as sc
import memento
import pandas as pd
import os
import numpy as np
from scipy.sparse import csr_matrix
from scipy.stats import false_discovery_control
os.chdir("/data/TAK_981")

In [ ]:
memento.

In [ ]:
memento.compute_1d_moments

Testing memento pipeline for single cond below

In [2]:
adata = sc.read_h5ad("h5ad/batch_corrected.h5ad")
adata = adata.raw.to_adata()
capture_rates = {
    "cd40-1": 0.2189,
    "cd40-2": 0.3719,
    "combo-1": 0.4790,
    "combo-2": 0.4405,
    "ctrl-1": 0.4015,
    "ctrl-2": 0.4047,
    "tak-1": 0.5386,
    "tak-2": 0.4268
}
adata.obs["capture_rate"] = adata.obs.sample_id.map(capture_rates).astype(int)*0.25

In [3]:
adata.X = adata.X.astype(np.float64)

In [8]:
comp = ["ctrlvcd40", "ctrlvtak", "ctrlvcombo", "takvcombo", "takvcd40", "cd40vcombo"]
celltypes = ["Epithelial", "Macrophages", "T-Cells"]
for i in celltypes:
    for n in comp:
        treat = n.split('v')
        ad_treat = adata[(adata.obs.condition.isin(treat)) &
                         (adata.obs.annot == i)]
        ad_treat.obs["treat"] = ad_treat.obs["condition"].apply(
            lambda x: 0 if x == treat[0] else 1)
        ad_treat.obs["samples"] = ad_treat.obs.sample_id.cat.codes
        ad_treat.X = csr_matrix(ad_treat.X.copy())
        memento.setup_memento(ad_treat, q_column="capture_rate")
        memento.create_groups(ad_treat, label_columns=['treat', "samples"])
        memento.compute_1d_moments(ad_treat)
        sample_meta = memento.get_groups(ad_treat)
        cov_df = pd.DataFrame(index=sample_meta.index)
        cov_df['intercept'] = 1
        memento.ht_1d_moments(ad_treat,
                              sample_meta,
                              covariate=cov_df,
                              num_boot=10000,
                              verbose=1,
                              num_cpus=15)
        result_1d = memento.get_1d_ht_result(ad_treat)\
        #    final_df.drop("Covariate", axis=1, inplace=True)
        result_1d = result_1d.loc[result_1d["tx"] == "treat"]
        padj_de = false_discovery_control(result_1d["de_pval"], axis=0)
        padj_dv = false_discovery_control(result_1d["dv_pval"], axis=0)
        result_1d.insert(result_1d.columns.get_loc('de_pval')+1, "de_padj", padj_de)
        result_1d.insert(result_1d.columns.get_loc('dv_pval')+1, "dv_padj", padj_dv)
        result_1d.to_csv(f"csv/de/memento/{i}_{n}.csv")

/tmp/ipykernel_4841/3157208910.py:8: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad_treat.obs["treat"] = ad_treat.obs["condition"].apply(
/home/ubuntu/miniconda3/envs/scanpy/lib/python3.10/site-packages/memento/main.py:181: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    7.2s
[Parallel(n_jobs=15)]: Done 170 tasks      | elapsed:   18.1s
[Parallel(n_jobs=15)]: Done 420 tasks      | elapsed:   37.5s
[Parallel(n_jobs=15)]: Done 770 tasks      | elapsed:  1.0min
[Parallel(n_jobs=15)]: Done 1220 tasks      | elapsed:  1.6min
[Parallel(n_jobs=15)]: Done 1770 tasks      | elapsed:  2.3min
[Parallel(n_jobs=15)]: Done 2420 tasks      | elapsed

In [ ]:
comp = ["ctrlvcd40", "ctrlvtak", "ctrlvcombo", "takvcombo", "takvcd40", "cd40vcombo"]
celltypes = ["Epithelial", "Macrophages", "T-Cells"]
for i in celltypes:
    for n in comp:
        treat = n.split('v')
        ad_treat = adata[(adata.obs.condition.isin(treat)) &
                         (adata.obs.annot == i)]
        ad_treat.obs["treat"] = ad_treat.obs["condition"].apply(
            lambda x: 0 if x == treat[0] else 1)
        ad_treat.obs["samples"] = ad_treat.obs.sample_id.cat.codes
        ad_treat.X = csr_matrix(ad_treat.X.copy())
        memento.setup_memento(ad_treat, q_column="capture_rate")
        memento.create_groups(ad_treat, label_columns=['treat'])
        memento.compute_1d_moments(ad_treat,
                                   min_perc_group=.7)
        sample_meta = memento.get_groups(ad_treat)
        cov_df = pd.DataFrame(index=sample_meta.index)
        cov_df['intercept'] = 1
        memento.ht_1d_moments(ad_treat,
                              sample_meta,
                              covariate=cov_df,
                              num_boot=10000,
                              verbose=1,
                              num_cpus=15)
        result_1d = memento.get_1d_ht_result(ad_treat)
        result_1d.to_csv(f"csv/de/memento_nosamp/{i}_{n}.csv")

In [ ]:
!sudo poweroff